In [1]:
# Oanda API teszter

In [2]:
## Account

In [3]:
import cfg

running price streamrunning transaction stream

running account update polling


In [4]:
cfg.account

In [5]:
for t in cfg.account.trades:
    print(f'{t.currentUnits:>7.0f}, {t.instrument}, {t.unrealizedPL:>8.4f}, {t.openTime}')

    367, EUR_USD,  -0.0440, 2022-02-03T20:41:39.169755239Z
   -372, EUR_CAD,  -0.1150, 2022-02-03T20:37:37.353441985Z
   -379, EUR_CHF,  -0.3556, 2022-02-03T20:25:31.067306015Z
    374, EUR_JPY,   0.0259, 2022-02-03T20:31:34.793314126Z
   -372, EUR_NZD,  -0.1344, 2022-02-03T20:37:38.056302892Z


In [6]:
print(f'{cfg.account.balance} nav:{cfg.account.NAV} margin:{cfg.account.marginAvailable}')

1516.6612 nav:1516.0381 margin:1460.4889


In [5]:
def check_breakeven_for_instrument(instrument: str, trades: list):
    all_breakeven = []
    for t in trades:
        if t.instrument == instrument:
            for o in cfg.account.orders:
                if o.id == t.stopLossOrderID:
                    all_breakeven.append(
                    (t.currentUnits > 0 ) and (o.price >=t.price)
                    or
                    (t.currentUnits <0 and o.price<=t.price))
    return all(all_breakeven)

In [6]:
check_breakeven_for_instrument('AUD_USD', cfg.account.trades)

False

In [7]:
for t in cfg.account.trades:
    print(t)
    break

id: '183576'
instrument: EUR_USD
price: 1.12933
openTime: '2022-01-26T14:29:13.285193386Z'
state: OPEN
initialUnits: -364.0
initialMarginRequired: 8.222
currentUnits: -364.0
realizedPL: 0.0
unrealizedPL: 0.5533
marginUsed: 8.2099
financing: 0.0
takeProfitOrderID: '183577'
stopLossOrderID: '183578'
trailingStopLossOrderID: '183579'


In [8]:
for o in cfg.account.orders:
    print(o)
    break

id: '183410'
createTime: '2022-01-26T09:38:07.001886322Z'
state: PENDING
type: TAKE_PROFIT
tradeID: '183409'
price: 0.71371
timeInForce: GTC
triggerCondition: DEFAULT


In [9]:
for t in cfg.account.trades:
    print(f'{t.instrument} {t.unrealizedPL}')

EUR_USD 0.5533
EUR_CHF -0.0121
USD_CHF 0.2413
AUD_USD 0.3439
AUD_JPY 0.4979
AUD_CAD -0.0643
EUR_GBP -0.0647


In [10]:
for p in cfg.account.positions:
    print(p)
    break

instrument: EUR_USD
pl: -388.9146
unrealizedPL: 0.6079
marginUsed: 8.2099
resettablePL: -388.9146
financing: -78.4801
commission: 0.0
guaranteedExecutionFees: 0.0
long:
  units: 0.0
  pl: -156.3461
  unrealizedPL: 0.0
  resettablePL: -156.3461
  financing: -79.6348
  guaranteedExecutionFees: 0.0
short:
  units: -364.0
  averagePrice: 1.12933
  tradeIDs:
  - '183576'
  pl: -232.5685
  unrealizedPL: 0.6079
  resettablePL: -232.5685
  financing: 1.1547
  guaranteedExecutionFees: 0.0


In [11]:
cfg.instruments['EUR_USD']

{'name': 'EUR_USD',
 'type': 'CURRENCY',
 'displayName': 'EUR/USD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'minimumTradeSize': '1.0',
 'maximumTrailingStopDistance': '1.0',
 'minimumTrailingStopDistance': '0.0005',
 'maximumPositionSize': '0.0',
 'maximumOrderUnits': '100000000.0',
 'marginRate': '0.02',
 'bid': 1.12766,
 'ask': 1.12779,
 'spread': 0.00013}